<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/%20%20%20%2005_Create_Vector_DB%20/Gemini/Create_Vector_Database_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Vector Database

In [ ]:
#installs
!pip install tqdm
!pip install -q sentence_transformers
!pip install -q pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 5.8 MB/s eta 0:00:00


In [ ]:
#imports
from tqdm import tqdm
import os
from google.colab import userdata
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from pinecone import Pinecone, PodSpec, ServerlessSpec
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Import Files From GitHub

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 1984, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 1984 (delta 232), reused 317 (delta 183), pack-reused 1595
Receiving objects: 100% (1984/1984), 366.42 MiB | 12.65 MiB/s, done.
Resolving deltas: 100% (1315/1315), done.
Updating files: 100% (1124/1124), done.


In [ ]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
text_df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN
...,...,...,...,...,...,...,...
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [ ]:
# Pull in sentiment analysis results
url =  '/content/Financial_Sentiment_LLM/03_Sentiment_Analysis/Prompt2_Sentiment_Analysis_Results.csv'
sentiment_df = pd.read_csv(url)
sentiment_df

,Source,Unique_ID,Ticker,Date,URL,Finance,Production,Reserves / Exploration / Acquisitions / Mergers / Divestments,Environment / Regulatory / Geopolitics,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price
0,Investment Research,IR-1,MRO,5/16/2024,NaN,Positive,Neutral,Positive,Negative,Neutral,Neutral
1,Investment Research,IR-2,EOG,5/14/2024,NaN,Positive,Neutral,Positive,Neutral,Positive,Neutral
2,Investment Research,IR-3,EOG,5/11/2024,NaN,Positive,Positive,Positive,Neutral,Neutral,Neutral
3,Investment Research,IR-4,DVN,5/11/2024,NaN,Positive,Neutral,Positive,Negative,Neutral,Neutral
4,Investment Research,IR-5,COP,5/7/2024,NaN,Positive,Neutral,Neutral,Negative,Neutral,Neutral
...,...,...,...,...,...,...,...,...,...,...,...
10120,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,NaN,Positive,Neutral,Negative,Positive,Positive,Neutral
10121,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,NaN,Positive,Positive,Neutral,Positive,Positive,Neutral
10122,Earnings Call Q&A,EQ-340,EOG,May-03-2019,NaN,Positive,Positive,Positive,Neutral,Neutral,Neutral
10123,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,NaN,Positive,Positive,Neutral,Neutral,Positive,Neutral


In [ ]:
# Merge text and sentiment dataframes
sentiment_df_subset = sentiment_df[["Unique_ID", "Alternative Energy / Lower Carbon",
                                    "Oil Price / Natural Gas Price / Gasoline Price",
                                    "Environment / Regulatory / Geopolitics",
                                    "Production"]]

# Drop NANs or articles where sentiment could not be determined
sentiment_df_subset = sentiment_df_subset.dropna()

# Merge the DataFrames on 'Unique_ID'
df_text_sentiment = pd.merge(text_df, sentiment_df_subset, on="Unique_ID", how='right')
df_text_sentiment

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price,Environment / Regulatory / Geopolitics,Production
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,Positive,Neutral,Neutral,Neutral
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,Neutral,Neutral,Neutral,Positive
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,Neutral,Neutral,Negative,Neutral
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,Neutral,Neutral,Negative,Neutral
...,...,...,...,...,...,...,...,...,...,...,...
10692,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Neutral
10693,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Positive
10694,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Positive
10695,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Neutral,Positive


In [ ]:
def chunk_text(text):
    """
    Splits a given text into chunks of sentences, each containing up to a specified maximum number of words.

    Parameters:
    text (str): The input text to be split into chunks.

    Returns:
    list: A list of text chunks, where each chunk is a string containing up to the specified maximum number of words.
    """
    try:
      max_word_count = 500
      sentences = sent_tokenize(text)
      chunks = []
      current_chunk = []
      current_word_count = 0

      for sentence in sentences:
          word_count = len(sentence.split())
          if current_word_count + word_count <= max_word_count:
              current_chunk.append(sentence)
              current_word_count += word_count
          else:
              chunks.append(' '.join(current_chunk))
              current_chunk = [sentence]
              current_word_count = word_count

      # Add the last chunk
      if current_chunk:
          chunks.append(' '.join(current_chunk))
    except:
      chunks = ['']

    return chunks

#Write chunks to dataframe
tqdm.pandas()
df_text_sentiment['Text Chunks'] = df_text_sentiment['Article Text'].progress_apply(chunk_text)
df_text_sentiment

100%|██████████| 10697/10697 [00:44<00:00, 241.93it/s]


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price,Environment / Regulatory / Geopolitics,Production,Text Chunks
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"[Stock Report | May 16, 2024 | NYSESymbol: MRO..."
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,Positive,Neutral,Neutral,Neutral,"[Stock Report | May 14, 2024 | NYSESymbol: EOG..."
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,Neutral,Neutral,Neutral,Positive,"[Stock Report | May 11, 2024 | NYSESymbol: EOG..."
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,Neutral,Neutral,Negative,Neutral,"[Stock Report | May 11, 2024 | NYSESymbol: DVN..."
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,Neutral,Neutral,Negative,Neutral,"[Stock Report | May 07, 2024 | NYSESymbol: COP..."
...,...,...,...,...,...,...,...,...,...,...,...,...
10692,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Neutral,[Question and Answer\nOperator\n[Operator Inst...
10693,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Positive,Positive,[Question and Answer\nOperator\n[Operator Inst...
10694,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Positive,[Question and Answer\nOperator\n[Operator Inst...
10695,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,Positive,Neutral,Neutral,Positive,[Question and Answer\nOperator\n[Operator Inst...


In [ ]:
#split into chunks and create new dataframe
df_chunk = df_text_sentiment.explode('Text Chunks').reset_index(drop=True)
df_chunk['Chunk_ID'] = df_chunk['Unique_ID'] + "-" + df_chunk.index.astype(str)
df_chunk = df_chunk.dropna(subset=['Text Chunks'])
df_chunk

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Alternative Energy / Lower Carbon,Oil Price / Natural Gas Price / Gasoline Price,Environment / Regulatory / Geopolitics,Production,Text Chunks,Chunk_ID
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",IR-1-0
1,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,2024 0.55 E 0.70 E 0.77 E 0.80 E 2.82\nproduct...,IR-1-1
2,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"2024 EBITDA, in line with MRO’s historical\nav...",IR-1-2
3,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"Chairman, President & CEOSenior VP of Technolo...",IR-1-3
4,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,Neutral,Neutral,Negative,Neutral,"Collectively, from 2017 through 2019,\n1,681\n...",IR-1-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82573,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,"We also,\nduring the quarter, completed the sa...",EQ-342-82573
82574,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,William L. Bullock\nPresident of Asia Pacific ...,EQ-342-82574
82575,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,Michael Anthony Hall\nHeikkinen Energy Advisor...,EQ-342-82575
82576,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,Neutral,Neutral,Neutral,Neutral,"And\nthat's one of the reasons, along with wha...",EQ-342-82576


In [ ]:
# Create Reference File for getting Chunks

# Determine the number of rows for each split
n = len(df_chunk)
split1 = int(n / 3)
split2 = int(2 * n / 3)

# Split the DataFrame
df1_chunk = df_chunk[:split1]
df2_chunk = df_chunk[split1:split2]
df3_chunk = df_chunk[split2:]

df1_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/Article_Chunk_References_pt1.csv', index=False)
df2_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/Article_Chunk_References_pt2.csv', index=False)
df3_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/Article_Chunk_References_pt3.csv', index=False)

In [ ]:

#Commit and Push Changes
!git config --global user.email "bw58@rice.edu"
%cd /content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/
!git add Article_Chunk_References_pt1.csv
!git commit -m "Add Article_Chunk_References_pt1.csv"
!git add Article_Chunk_References_pt2.csv
!git commit -m "Add Article_Chunk_References_pt2.csv"
!git add Article_Chunk_References_pt3.csv
!git commit -m "Add Article_Chunk_References_pt3.csv"
!git push origin main


/content/Financial_Sentiment_LLM/05_Create_Vector_DB
[main 5cf4bc7] Add Article_Chunk_References_pt1.csv
 1 file changed, 726 deletions(-)
[main 9971c39] Add Article_Chunk_References_pt2.csv
 1 file changed, 726 insertions(+), 91 deletions(-)
[main 989abd5] Add Article_Chunk_References_pt3.csv
 1 file changed, 91 insertions(+), 71 deletions(-)
Enumerating objects: 17, done.
Counting objects: 100% (17/17), done.
Delta compression using up to 2 threads
Compressing objects: 100% (12/12), done.
Writing objects: 100% (12/12), 34.78 KiB | 8.69 MiB/s, done.
Total 12 (delta 9), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (9/9), completed with 5 local objects.
remote: warning: See https://gh.io/lfs for more information.
remote: warning: File 05_Create_Vector_DB/Article_Chunk_References_pt1.csv is 71.05 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: File 05_Create_Vector_DB/Article_Chunk_References_pt2.csv is 72.42 MB; this is lar

### Encode Chunks

In [ ]:
#create text embeddings of chunks
embedding_model = SentenceTransformer("all-mpnet-base-v2")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model.to(device)
text_embeddings = embedding_model.encode(df_chunk['Text Chunks'].tolist())
text_embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(82507, 768)

### Create Vector Database

In [ ]:
#create vector
vectors = []
for idx in range(text_embeddings.shape[0]):
  try:
    vector = [float(i) for i in list(text_embeddings[idx])]
    vectors.append({"id": str(df_chunk['Chunk_ID'][idx]),
                    "metadata": {"date": str(df_chunk['Date'][idx]),
                                "alt_energy": df_chunk['Alternative Energy / Lower Carbon'][idx],
                                "oil_price": df_chunk['Oil Price / Natural Gas Price / Gasoline Price'][idx],
                                "env_reg": df_chunk['Environment / Regulatory / Geopolitics'][idx],
                                "prod": df_chunk['Production'][idx],
                                "ticker": df_chunk['Ticker'][idx]},
                    "values": vector })
  except:
    pass

#vectors[text_embeddings.shape[0]-1]

In [ ]:
#delete existing index to rebuild with new data
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

pc.delete_index("fastvectors")

In [ ]:
#create vector database index
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

index_name = "fastvectors"

pc.create_index(
    name=index_name,
    dimension=text_embeddings.shape[1], # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
# insert vectors into database
index = pc.Index(index_name)

# segments set to batch into vector database because of 2MB send limit
segments = 20
sections = int(text_embeddings.shape[0] / segments)
for i in tqdm (range(sections+1), desc="Loading..."):
  index.upsert(
    vectors=vectors[segments*i:segments*(1+i)],
    namespace="ns1"
  )

Loading...: 100%|█████████▉| 4122/4126 [25:42<00:01,  2.67it/s]


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 19 Jun 2024 05:44:47 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1', 'x-pinecone-request-id': '7197779163573193237', 'x-envoy-upstream-service-time': '1', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [ ]:
#check index
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 84436}},
 'total_vector_count': 84436}

In [ ]:
from google.colab import runtime
runtime.unassign()